😀 😃 😄 😁 😆 😅 😂 🤣 😊 😇 🙂 🙃

**Автор:** Миша

**Цель:** посмотреть, как работают и как встраиваются в пайплайн разные штуки для отбора переменных.

**Библиотеки:** `feature_engine`, `mlxtend`

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline

Будем использовать следующие модели:

In [2]:
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

Воспользуемся для примера первым датасетом (German).

In [3]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data',
                 header = None, sep = ' ')

# based on the .doc data description
df.columns = ['cheq_acc', 'dur_t', 'cred_hist', 'purp', 'cred_amt', 'save_acc', 
              'empl_t', 'inst_to_income', 'pers_status', 'guarant_flg',
              'residence_t', 'prop', 'age', 'inst_plan', 'house', 'n_loans',
              'job', 'n_depend', 'tel_flg', 'foreign_flg', 'target']

cat_vals = ['cheq_acc', 'cred_hist', 'purp', 'save_acc', 
            'empl_t', 'pers_status', 'guarant_flg', 'prop', 
            'inst_plan', 'house', 'job', 'tel_flg', 'foreign_flg']
num_vals = ['dur_t', 'cred_amt', 'inst_to_income', 'residence_t', 
            'age', 'n_loans', 'n_depend']

Применим WoE-преобразование к категориальным фичам:

In [4]:
from feature_engine.encoding import WoEEncoder
encoder = WoEEncoder(variables=cat_vals)

X = df.drop("target", axis=1)
y = df["target"] - 1
encoder.fit(X, y)

X = encoder.transform(X)

# Greedy (backward & forward) selection

## Как работает

Не понял, почему в эксель-файле для greedy-selection стоит `feature_engine`: такой функциональности в нем не нашел.

Нашим инструментом здесь будет `SequentialFeatureSelector`. [Гайд](http://rasbt.github.io/mlxtend/user_guide/feature_selection/SequentialFeatureSelector/#overview).

In [2]:
from mlxtend.feature_selection import SequentialFeatureSelector

В датасете у нас 20 фичей, хотим выбрать 5 лучших с помощью greedy-selection:

In [52]:
selection = SequentialFeatureSelector(
    estimator=LogisticRegression(penalty="none", max_iter=1000),  # базовая модель
    k_features=5,                                                 # сколько фичей хотим в итоге
    forward=True,                                                 # как отбираем: от нуля - forward или ото всех - backward
    floating=True,                                                # исключаем ли переменные
    verbose=1,
    cv=5
)

selection.fit(X, y)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.7s finished
Features: 1/5[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
Features: 2/5[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:    0.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished
Features: 3/5[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  17 out of  17 | elapsed:    1.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend w

SequentialFeatureSelector(estimator=LogisticRegression(max_iter=1000,
                                                       penalty='none'),
                          floating=True, k_features=5, verbose=1)

В целом, по аргументам можно видеть все опции, которые перед нами есть. Поэтому можем рассматривать `mlxtentions` как альтернативу `feature_engine` для feature selection.

Названия отобранных фичей:

In [56]:
selection.k_feature_names_

('cheq_acc', 'dur_t', 'cred_hist', 'save_acc', 'empl_t')

Еще может остаться вопрос - по какой метрике этот алгоритм сравнивает модели. Классификаторы - по аккураси, но можно реализовать и свою. Сделаем скоринг случайным числом! (с помощью инфы [отсюда](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html)).

In [66]:
from sklearn.metrics import make_scorer

@make_scorer
def random_metric(y, y_pred):
    return np.random.rand()

In [72]:
random_selection = SequentialFeatureSelector(
    estimator=LogisticRegression(penalty="none", max_iter=1000),
    k_features=3,                                                 
    forward=True,                                                 
    floating=True,                                                
    scoring=random_metric,  # : )))
    verbose=1,
    cv=5
)

random_selection.fit(X, y)

print(f"Самые крутые фичи: {random_selection.k_feature_names_}")

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.6s finished
Features: 1/3[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
Features: 2/3[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:    1.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished
Features: 3/3

Самые крутые фичи: ('cred_hist', 'age', 'job')


## Как встроить в пайплайн

Метод `fit` есть (выше), `transform` тоже:

In [59]:
selection.transform(X)

array([[ 8.18098706e-01,  6.00000000e+00, -7.33740578e-01,
        -7.04246074e-01, -2.35566071e-01],
       [ 4.01391783e-01,  4.80000000e+01,  8.83186170e-02,
         2.71357844e-01,  3.21032454e-02],
       [-1.17626322e+00,  1.20000000e+01, -7.33740578e-01,
         2.71357844e-01, -3.94415272e-01],
       ...,
       [-1.17626322e+00,  1.20000000e+01,  8.83186170e-02,
         2.71357844e-01, -2.35566071e-01],
       [ 8.18098706e-01,  4.50000000e+01,  8.83186170e-02,
         2.71357844e-01,  3.21032454e-02],
       [ 4.01391783e-01,  4.50000000e+01, -7.33740578e-01,
         1.39551880e-01,  3.19230430e-01]])

Попробуем встроить в пайплайн: прикол тут еще в том, что фичи отбираются на основе бустинга, а в итоге на них учится логрег. Да, так тоже можно :)

In [102]:
selection = SequentialFeatureSelector(
    estimator=XGBClassifier(use_label_encoder=False, eval_metric="logloss"),  
    k_features=5,                                                  
    forward=True,                                                  
    floating=True,                                                
    verbose=0,
    cv=5
)

pipeline = Pipeline(
    [
        ('selection', selection),
        ('model', LogisticRegression(penalty="none", max_iter=1000))
    ]
)

pipeline.fit(X, y);

Что еще есть интересного по теме в `mlxtentions`:

- `ColumnSelector` можно использовать как часть `GridSearch`
- `ExhaustiveFeatureSelector` по названию все ясно :)

# Stepwise (backward) без переобучения

Перейдем теперь к `feature_engine`. В ноутбуке `missing_values` есть пример использования `RecursiveFeatureAddition`. Мы же рассмотрим `SelectByShuffling` -- способ отбора фичей без переобучения модели:

In [5]:
from feature_engine.selection import SelectByShuffling

Будем шафлить фичи и выкидывать их, если в результате аук снижается менее чем на 0.01:

In [90]:
selection = SelectByShuffling(
    estimator=LogisticRegression(penalty="none", max_iter=1000),
    variables=X.columns.to_list(),                                      # можно задать подмножество
    scoring='roc_auc',                                                  # метрика
    threshold=0.01,                                                     # порог ее снижения
    cv=5,
    random_state=98
)

selection.fit(X, y)

SelectByShuffling(cv=5,
                  estimator=LogisticRegression(max_iter=1000, penalty='none'),
                  random_state=98, threshold=0.01,
                  variables=['cheq_acc', 'dur_t', 'cred_hist', 'purp',
                             'cred_amt', 'save_acc', 'empl_t', 'inst_to_income',
                             'pers_status', 'guarant_flg', 'residence_t',
                             'prop', 'age', 'inst_plan', 'house', 'n_loans',
                             'job', 'n_depend', 'tel_flg', 'foreign_flg'])

Итак, можно удалить 18 фичей:

In [91]:
len(selection.features_to_drop_)

18

Метод `transform` есть, в пайплайн встанет.

In [94]:
selection.transform is not None

True

## Разбор случая, когда `SelectByShuffling` отбрасывает все фичи.

Сгенерим датасет, где признаки и таргет никак не связаны.

In [18]:
np.random.seed(89)
X = pd.DataFrame(
    np.random.randn(10000, 10),
    columns=[f"trash_feature_{i}" for i in range(10)]
)
y = np.random.randint(2, size=10000)

In [19]:
selection = SelectByShuffling(
    estimator=LogisticRegression(penalty="none", max_iter=1000),
    scoring='roc_auc',                                                  # метрика
    threshold=0.01,                                                     # порог ее снижения
    cv=5,
    random_state=98
)

selection.fit(X, y)

SelectByShuffling(cv=5,
                  estimator=LogisticRegression(max_iter=1000, penalty='none'),
                  random_state=98, threshold=0.01)

Все фичи дропнулись:

In [20]:
len(selection.features_to_drop_)

10

В таком случае `transform` возвращает пустой датафрейм:

In [22]:
selection.transform(X).shape

(10000, 0)

Обработаем этот случай следующим образом: добавим возможность задавать нижнюю границу количества фичей, которую должен вернуть `SelectByShuffling`. Если этот параметр равен 5, а отбор прошло только 3 признака, то вернутся 5 лучших. По умолчанию этот параметр равен 1, что позволит обработать случай 0 отобранных фичей.

In [57]:
class SafeSelectByShuffling(SelectByShuffling):

    def __init__(self, *args, min_features=1, **kwargs):
        super().__init__(*args, **kwargs)
        self.min_features = min_features

    def transform(self, X):

        n_features_left  = self.n_features_in_ - len(self.features_to_drop_)
        m = self.min_features

        if n_features_left >= self.min_features:
            return super().transform()

        else:
            print((
                f"Less than min_features = {m} are left, "
                f"return {m} best feature{'' if m == 1 else 's'} by performance drift."
                ))
            features, drifts = zip(*self.performance_drifts_.items())                     # разобьем словарь на ключи и значения
            features = np.array(features)[np.argsort(drifts)[::-1]]                       # отсортируем названия фичей по убыванию изменения метрики
            return X[features[:self.min_features]]                                        # возвращаем self.min_features признаков с наилучшими значениями метрики


Пример работы:

In [58]:
selection = SafeSelectByShuffling(
    estimator=LogisticRegression(penalty="none", max_iter=1000),
    scoring='roc_auc',                                                  
    threshold=0.01,                                                    
    cv=5,
    random_state=98,
    min_features=5
).fit(X, y)

selection.transform(X)

Less than min_features = 5 are left, return 5 best features by performance drift.


,trash_feature_4,trash_feature_2,trash_feature_9,trash_feature_7,trash_feature_5
0,-0.305779,-0.425892,-0.879576,-1.202224,0.040550
1,-0.206452,0.344946,0.133677,0.398148,-0.871597
2,0.469517,-0.546566,-0.615288,0.418356,-2.091232
3,-1.782605,-0.802347,1.379050,-1.313043,0.741371
4,2.292134,0.163758,0.379371,0.171874,0.664245
...,...,...,...,...,...
9995,0.021130,0.038834,-0.270902,0.459490,0.424119
9996,-1.838206,-0.175155,0.925820,0.225701,-0.827336
9997,0.710940,-0.057501,-0.648435,-0.156609,-1.741182
9998,-0.189659,0.491383,0.261676,1.060973,-0.359844


# SmartCorrelatedSelection

Прикольная по идее штука, вся нужная инфа в [документации](https://feature-engine.readthedocs.io/en/1.1.x/selection/SmartCorrelatedSelection.html).

In [96]:
from feature_engine.selection import SmartCorrelatedSelection

Опять можно выбрать подмножество признаков, а вот модель уже **не нужна**!

In [97]:
selection = SmartCorrelatedSelection(
    variables=X.columns.to_list(),
    method="pearson",                # можно взять свою функцию
    threshold=0.3,                   # порог корреляции
    selection_method="variance",     # из коррелирующих групп выбираем признак с наиб дисперсией
    estimator=None,                  # понадобится для selection_method="model_performance"        
    cv=5, 
)

selection.fit(X)

SmartCorrelatedSelection(cv=5, selection_method='variance', threshold=0.3,
                         variables=['cheq_acc', 'dur_t', 'cred_hist', 'purp',
                                    'cred_amt', 'save_acc', 'empl_t',
                                    'inst_to_income', 'pers_status',
                                    'guarant_flg', 'residence_t', 'prop', 'age',
                                    'inst_plan', 'house', 'n_loans', 'job',
                                    'n_depend', 'tel_flg', 'foreign_flg'])

Группы коррелирующих фичей:

In [98]:
selection.correlated_feature_sets_

[{'cred_amt', 'dur_t'},
 {'cred_hist', 'n_loans'},
 {'house', 'residence_t'},
 {'job', 'tel_flg'}]

Какие нужно выкинуть:

In [100]:
selection.features_to_drop_

['dur_t', 'cred_hist', 'house', 'tel_flg']

Встраиваем в пайплайн:

In [101]:
pipeline = Pipeline(
    [
        ('selection', selection),
        ('model', LogisticRegression(penalty="none", max_iter=1000))
    ]
)

pipeline.fit(X, y);

Кстати, чтобы сделать предикт только для наблюдения 2, нужно пихать данные в пайплайн вот в такой странной форме)

In [130]:
pipeline.predict(X.iloc[1:2, :])

array([0], dtype=int64)

Выводы:
- есть прикольная библиотека `mlxtend` где реализовано greedy selection (и другие более простые алгоритмы forward- и backward- selection)
- из `feature_engine` можно взять `SelectByShuffling` для отбора фичей без переобучения
- в обеих библиотеках есть другие прикольные штуки, например `SmartCorrelatedSelection` из `feature_engine`, которая позволяет находить группы коррелированных (по произвольной метрике!) фичей и выбирать из каждой группы одну (на основе произвольного критерия!).